# Vislice
V tej nalogi si bomo pogledali slovenske samostalnike s strani igranja igre vislice. Pogledali si bomo predvsem kako izgledajo optimalne strategije in kakšne besede
so vsaj v teoriji najtežje.

Po izbiri pripravite pythonovo virtualno okolje. Preden uporabljate katerekoli programe v tem repozitoriju, poženite naslednje ukaze:

In [ ]:
!pip install -r requirements.txt
!python data/get_lf_no_lfs.py


## Natančnejši opis problema
### Pravila igre
V splošnem se igro igrata dve osebi. Ena oseba si izmisli besedo, druga pa ugiba črke. Vsakič ko črko pravilno ugotovi, se črko zapiše na njena mesta v besedi.
Če beseda ugibane črke ne vsebuje, oseba ki ugiba izgubi življenje. To se ponavadi označi tako, da druga oseba nariše eno črto na skico obešenega človeka.
Ko je slika končana, je igre konec in zmaga tisti, ki si je besedo izmislil. Če ugotovi vse črke v besedi, zmaga tisti, ki ugiba.

To je splošen opis igre, ki še vedno dopušča mnogo variacij. Za potrebe te naloge se bomo odločili za nekaj točnejših pravil:
* Izbrana beseda je na seznamu besed, ki je znan vnaprej, to so vsi samostalniki iz SSKJ, ki vsebujejo le črke slovenske abecede. Shranjeni so v `data/nouns_si.txt`.
* Male in velike črke uporabljamo za iste črke.
* Ko ugibamo, na začetku poznamo dolžino besede, prvo črko in kje vse se ta črka ponovi.
* Z vidika naloge je število poskusov ko lahko ugibamo nepomembno, zato števila življenj ne bomo določili.

### Optimalna strategija
Kaj je optimalna se lahko spreminja glede na naše potrebe. Za naše potrebe bomo iskali takšno optimalno strategijo:
* Z vsako strategijo bomo pri eni ali več besedah porabili največje število življenj. Če ugibamo to besedo je to najslabši možen primer.
* Strategija je optimalna, če ima v primerjavi z ostalimi strategijami po njihovih najslabših možnih primerih najmanjše število porabljenih življenj.
* Opazovali bomo samo najslabši možen primer. Če imata dve strategiji enako slab najslabši primer, sta enako dobri. Tudi če ena od strategij vse ostale besede reši
z manj izgubljenimi življenji, sta še vedno enako dobri.
* Za strategijo nas zanima število izgubljenih življenj da uganemo besedo v najslabšem možnem primeru. Zato v pravilih ni smiselno določiti po koliko napakah je igre konec.
* Optimalnih strategij je lahko več. Ponavadi jih je celo zelo veliko. 
(Če imamo "se_" - prvi črki sta "s" in "e", ugibamo še tretjo, potem imamo na voljo 10 črk ki dajo besedo in 3628800 različnih strategij)

### Seznam besed
Vir besed je stran [https://fran.si/iskanje?FilteredDictionaryIds=130&View=1&Query=%2A](https://fran.si/iskanje?FilteredDictionaryIds=130&View=1&Query=%2A). 
Program `src/collector/sskj_collector.py` prebere število strani z besedami in pobere besede z vsake izmed njih. Ima tudi možnost, da se omeji samo na samostalnike.
Potem popravi še vsa naglasna znamenja v navadne črke in pobriše besede ki ne vsebujejo samo slovenskih črk. Na koncu jih uredi po slovenski abecedi.

Besede so že shranjene v datoteki [data/nouns_si.txt](../data/nouns_si.txt), zato jih ni potrebno ponovno nalagati.
Če želite pognati pobiranje besed lahko to storite s pomočjo skripte z ukazom (postopek traja, ker prebere več kot 4600 strani):

In [115]:
%run vislice.py sskjcollect --help

usage: vislice.py sskjcollect [-h] [--limit LIMIT] [--threads THREADS]
                              [--noprogress] [--file FILE] [--raw]
                              [--nounsonly]

options:
  -h, --help         show this help message and exit
  --limit LIMIT      limit the number of pages to collect words from (default:
                     all)
  --threads THREADS  maximum number of threads to use (default: 100)
  --noprogress       show progress while collecting words (default)
  --file FILE        file to save words to (default: data/sskj_words.txt)
  --raw              by default it removes accents, special characters, ...
  --nounsonly        collect only nouns


In [ ]:
%run vislice.py sskjcollect --file data/samostalniki.txt --nounsonly

### Izračun optimalne strategije
Glede na podan seznam besed program `src/solver/game.py` izračuna najboljšo možno strategijo tako, da sledi korakom:
* Če se v že ugibanih črkah besede ne ujemajo, jih razdelimo na skupine, tako da glede na te črke izgledajo enako. Za vsako skupino izračunamo strategijo posebej in naša strategija ima
toliko napak kot jih ima skupina z največ napakami.
* Če smo besedo uganili, smo zanjo že zmagali. Strategija ima 0 korakov in naredimo 0 napak.
* Če si vse možne besede delijo kakšno črko, ugibamo te črke. Besede razdelimo v skupine, tako da imajo ugibane črke na istih mestih in za vsako od dobljenih skupin
izračunamo najboljšo strategijo. Strategija ima toliko napak kot skupina z največ napakami.
* Za vsako črko, ki je še nismo ugibali, od najbolj do najmanj pogoste poskusimo:
    - Besede razdelimo v skupine, tako da se ujemajo v vseh do zdaj ugibanih črkah in v tej izbrani črki.
    - Izračunamo strategijo za vse skupine. Če besede v tej skupini ne vsebujejo izbrane črke, ima skupina eno napako več kot njena strategija.
    Če besede v skupini vsebujejo črko, ima skupina toliko napak kot ima napak njena strategija.
    - Za najboljšo strategijo vzamemo črko, ki ima pri najslabši skupini najmanjše število napak.
Na koncu dobimo eno izmed najboljših strategij. Strategija je že izračunana in shranjena v [data/nouns_si_strategy_hinted.json](../data/nouns_si_strategy_hinted.json)

Če želite poračunati strategijo lahko to storite s pomočjo ukaza spodaj. (Zelo traja - lahko tudi več kot 8 ur, poraba pomnilnika je do 3 GB.) Spremenite `data/nouns_si.txt`, če želite drug seznam besed.

In [116]:
%run vislice.py hintstrat --help

usage: vislice.py hintstrat [-h] [--output OUTPUT] [--limit LIMIT] words

positional arguments:
  words            file with words to find strategy for

options:
  -h, --help       show this help message and exit
  --output OUTPUT  file to save strategy to (default: data/hstrategy.json)
  --limit LIMIT    limit the number of words to find strategy for (default:
                   all)


In [ ]:
%run vislice.py hintstrat data/nouns_si.txt --output data/strategija.json

## Analiza podatkov


In [117]:
import pandas as pd
import json

In [118]:
with open("data/nouns_si_strategy_hinted.json", encoding="utf-8") as f:
    strategy_object = json.load(f)

words = pd.array(strategy_object["words"])
strategies = strategy_object["strategies"]

Za začetek preberemo seznam dovoljenih besed:

In [119]:
words

<StringArray>
[        'abak', 'abalienacija',         'abbe',      'abderit',
  'abderitstvo',   'abdikacija',      'abdomen',    'abdukcija',
      'abeceda',     'abecedar',
 ...
      'žvižgač',    'žvižganje',    'žvižgavec',    'žvižgavka',
       'žvokno',   'žvrgolenje',  'žvrgolevček',      'žvrkelj',
      'žvrklja',   'žvrkljanje']
Length: 49909, dtype: string

Za vsako besedo vzamemo prvo črko in pogledamo kje vse se ponovi. Tako dobimo vzorec, ki mu lahko sledi ena ali več besed. Vsak od takih vzorcev nam predstavlja isto izhodišče za igranje igre. Tukaj se naložijo osnovni podatki za takšne skupine.

In [120]:
strategies_info = pd.json_normalize(
    [
        {"pattern": " ".join(key), "words_count": len(value["words"]), "worst_case_mistakes": value["max_errors"]}
        for key, value in strategies.items()
    ]
)
strategies_info

,pattern,words_count,worst_case_mistakes
0,a _ a _,6,4
1,a _ a _ _ _ _ a _ _ _ a,1,0
2,a _ _ _,30,5
3,a _ _ _ _ _ _,80,3
4,a _ _ _ _ _ _ _ _ _ _,30,1
...,...,...,...
2412,ž _ ž _ _ _ _ _ _ _ _,3,0
2413,ž _ ž _ _ _ _ _ _,2,0
2414,ž _ _ ž _,1,0
2415,ž _ _ ž _ _ _,1,0


S temi podatki o strategijah lahko odgovorimo na dve zelo zanimivi vprašanji: 
- Kdaj (če nimamo sreče) lahko pričakujemo da se bomo zmotili največkrat?
- Kdaj lahko zmagamo brez da se zmotimo?

In [121]:
worst = strategies_info[strategies_info["worst_case_mistakes"] == strategies_info["worst_case_mistakes"].max()]
worst

,pattern,words_count,worst_case_mistakes
1023,k _ _ _,117,10
1159,l _ _,35,10


Iz zgornje tabele lahko vidimo da sta najslabša možna začetka "k _ _ _" in "l _ _". Gre za skupine relativno kratkih besed, kar je pričakovano, saj poznamo 1. črko, izmed ostalih 24 v abecedi pa moramo pravilno uganiti 2 ali 3. Če o besedah ne bi vedeli ničesar, bi lahko v takšnem primeru pričakovali 22 ali pa 21 napak. Zdaj vemo tudi, da če imamo na voljo vsaj 11 življenj, lahko igro vislic zmagamo vedno (pri postavljenih pravilih).

Najboljših možnih začetkov je več. 

In [122]:
best = strategies_info[strategies_info["worst_case_mistakes"] == strategies_info["worst_case_mistakes"].min()]
print(f"Number of best possible starts: {len(best["words_count"])}")
print(f"Number of words that give a best possible start: {best["words_count"].sum()}")
print(f"Number of starts where only one word is possible: {len(best[best["words_count"] == 1])}")

best[best["words_count"] > 1].reset_index()

Number of best possible starts: 1523
Number of words that give a best possible start: 3272
Number of starts where only one word is possible: 861


,index,pattern,words_count,worst_case_mistakes
0,22,a _ _ _ _ _ _ _ _ _ _ _,20,0
1,30,a _ _ _ _ _ _ _ _ a _ _ _ a,7,0
2,32,a _ _ _ _ _ _ _ a _ _ _,7,0
3,35,a _ _ _ _ a _ _ _ _ _,13,0
4,36,a _ _ _ _ a _ _ _ _ _ _,2,0
...,...,...,...,...
657,2407,ž _ _ _ _ _ _ _ _ _ _ _ _ _ _,3,0
658,2410,ž _ ž _ _ _ _,2,0
659,2412,ž _ ž _ _ _ _ _ _ _ _,3,0
660,2413,ž _ ž _ _ _ _ _ _,2,0


Iz zgornje celice dobimo odgovor na drugo vprašanje. Brez da se zmotimo lahko zmagamo v 1523 primerih, od tega lahko v 861 primerih takoj uganemo besedo. Prej smo že ugotovili, da imamo pri izbranem seznamu 49909 besed 2417 možnih začetkov. Garantirano zmago nam da 3272 besed, ampak to ne pomeni, da je to število besed ki jih lahko uganemo brez napak. Z uporabo izračunane strategije pri vsakem začetku lahko nekatere besede ugotovimo brez napak. Izračunati čim več besed brez da kdaj zgrešimo črko ni bilo postavljeno merilo za optimalno strategijo, zato te statistike nima smisla vključiti v rezultate, saj bi lahko imeli drugo optimalno strategijo, ki bi dala drugačne rezultate.

### Slabi začetki
Zdaj že vemo kakšna sta najslabša možna začetka. Če igramo vislice proti nekomu in smo mi izbrali besedo, nam pa to še ne pomaga preveč. Zanima nas, katere besede si moramo izbrati, da bo naš nasprotnik najbolj trpel, tudi če je robot in igra optimalno.

In [123]:
worst_words1 = strategies["l__"]["words"]
print(worst_words1)
worst_words2 = strategies["k___"]["words"]
print(worst_words2)

['laj', 'lak', 'lan', 'lar', 'las', 'lat', 'laz', 'laž', 'led', 'lej', 'lek', 'lep', 'les', 'leš', 'let', 'lev', 'lij', 'lik', 'lim', 'liv', 'lob', 'loč', 'log', 'loj', 'lok', 'lom', 'lon', 'los', 'lot', 'lov', 'lož', 'lub', 'luč', 'lug', 'luk']
['kača', 'kadi', 'kaja', 'kala', 'kalo', 'kamp', 'kana', 'kanu', 'kaos', 'kapa', 'kapo', 'kare', 'karo', 'karp', 'kart', 'kasa', 'kaša', 'kava', 'kavč', 'kavs', 'keha', 'kela', 'kepa', 'kila', 'kilt', 'kinč', 'kino', 'kist', 'kita', 'kivi', 'klan', 'klas', 'kleč', 'klej', 'klen', 'klep', 'kler', 'klet', 'klic', 'klin', 'klob', 'klon', 'klop', 'klor', 'klot', 'klub', 'kmet', 'knap', 'knez', 'knof', 'koca', 'koča', 'koda', 'kofe', 'koja', 'kola', 'kolc', 'kolč', 'kolo', 'kolt', 'koma', 'komi', 'konj', 'kopa', 'kord', 'korm', 'kosa', 'kost', 'kota', 'koza', 'koža', 'krah', 'kraj', 'krap', 'kras', 'krat', 'kreč', 'kreg', 'krep', 'kres', 'krez', 'krič', 'križ', 'krlj', 'krma', 'krof', 'krog', 'kroj', 'krom', 'krop', 'kros', 'krov', 'krpa', 'krst', '

Niso pa vse te besede najzlobnejša izbira. Natančno si poglejmo zakaj in katere izmed teh besed se splača izbrati.

In [124]:
def group_by_letter(words, letter_indices):
    groups = pd.DataFrame(words).groupby(lambda i: tuple(words[i][j] for j in letter_indices)).groups
    maxlen = max(map(len, groups.values()))
    return pd.DataFrame(
        ([words[group.values[i]] if len(group.values) > i else "" for group in groups.values()] for i in range(maxlen)),
        columns=map(" ".join, groups.keys()),
    )


group_by_letter(worst_words1, letter_indices=(1,))

,a,e,i,o,u
0,laj,led,lij,lob,lub
1,lak,lej,lik,loč,luč
2,lan,lek,lim,log,lug
3,lar,lep,liv,loj,luk
4,las,les,,lok,
5,lat,leš,,lom,
6,laz,let,,lon,
7,laž,lev,,los,
8,,,,lot,
9,,,,lov,


In [125]:
df = group_by_letter(worst_words2, letter_indices=(1, 3))
df.loc[:, df.iloc[2] != ""]  # Too many columns, select only those with at least 3 words.

,a a,a o,e a,l n,o a,r j,r p,r s,u a
0,kača,kalo,keha,klan,koca,kraj,krap,kras,kuba
1,kaja,kapo,kela,klen,koča,krlj,krep,kres,kuga
2,kala,karo,kepa,klin,koda,kroj,krop,kros,kuha
3,kana,,,klon,koja,,krup,,kula
4,kapa,,,,kola,,,,kuma
5,kasa,,,,koma,,,,kuna
6,kaša,,,,kopa,,,,kupa
7,kava,,,,kosa,,,,kura
8,,,,,kota,,,,kuta
9,,,,,koza,,,,kuža


S tako zapisanimi besedami je precej očitno zakaj v tej skupini ne moremo v najslabšem primeru porabiti manj kot 10 poskusov. V prvi tabeli so zares najslabše besede besede v stolpcu "o" - tiste, ki se začnejo na "lo", v drugi tabeli pa besede v stolpcu "oa". Tema seznamoma besed je skupno, da imata 11 različnih besed, ki se razlikujejo le v eni črki na enem mestu. V najslabšem primeru se bomo 10-krat zmotili in glede tega ne moremo storiti ničesar.